In [ ]:
# C11. Monitorer un modèle d’intelligence artificielle à partir des métriques courantes et spécifiques au projet,
# en intégrant les outils de collecte, d’alerte et de restitution des données du monitorage pour permettre
# l’amélioration du modèle de façon itérative.


In [ ]:
# Pour mettre en place le suivi des trois métriques sélectionnées (similarité moyenne, écart-type de similarité, 
# popularité moyenne des recommandations), voici les étapes détaillées que j'ai suivi pour leur calcul, collecte, et envoi à Prometheus.

In [2]:
# 1 Biblio necessaires
%pip install prometheus_client flask


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 2 Config de Prometheus dans app.py (API)


# Creation du fichier prometheus.yml 

#                                                   lancement avec docker :

# docker run -d --name=prometheus -p 9090:9090 `
#     -v "C:\Users\User\Downloads\CoursAlternance\Chefoeuvre\RecommendationsLectures\prometheus.yml:/etc/prometheus/prometheus.yml" `
#     prom/prometheus



#                                                 Interface Prometheus

# http://localhost:9090

# docker stop prometheus

# et 


# docker start prometheus

#PERMET D'ARRETER ET RELANCER POUR FAIRE DES MODIFS



#                           Ajout des imports et configuration du client Prometheus dans  app.py 

# from prometheus_client import Gauge, Histogram, start_http_server
# import numpy as np

# # Démarrage du serveur Prometheus sur le port 8000
# start_http_server(8000)

# # Création des métriques Prometheus
# similarity_mean_gauge = Gauge('recommendation_similarity_mean', 'Mean similarity score of recommendations')
# similarity_std_gauge = Gauge('recommendation_similarity_std', 'Standard deviation of similarity scores for recommendations')
# popularity_mean_gauge = Gauge('recommendation_popularity_mean', 'Mean popularity rating of recommended books')




In [ ]:
# Verifier que le port 8000 est libre

# netstat -aon | findstr :8000


In [ ]:
# 3 Calcul et enregistrement des métriques dans get_recommendations

# Dans la fonction get_recommendations, où les recommandations sont générées,
# On va calculer et collecter les métriques. l'implémentation pour les trois métriques est la suivante :

# @app.route('/recommendations', methods=['POST'])
# @jwt_required()
# def get_recommendations():
#     user_input = request.json.get('book_title', '')
#     if user_input:
#         recommendations = recommander_livres_sans_categorie(user_input, data, cosine_sim_embeddings)
        
#         # Calcul des métriques de similarité
#         similarity_scores = [rec['score'] for rec in recommendations]
#         similarity_mean = np.mean(similarity_scores)
#         similarity_std = np.std(similarity_scores)
        
#         # Enregistrement des métriques de similarité dans Prometheus
#         similarity_mean_gauge.set(similarity_mean)
#         similarity_std_gauge.set(similarity_std)

#         # Calcul de la popularité moyenne des recommandations
#         popularity_scores = [rec['average_rating'] for rec in recommendations if 'average_rating' in rec]
#         if popularity_scores:
#             popularity_mean = np.mean(popularity_scores)
#             popularity_mean_gauge.set(popularity_mean)
        
#         return jsonify(recommendations)
#     return jsonify({"msg": "No book title provided"}), 400




In [ ]:
# 4 exposition des métriques à Prometheus 

# Dès que l’API app.py est lancée,
# Prometheus commence à collecter les données exposées sur le port 8000. Les métriques seront visibles via cette URL :
# http://localhost:8000/metrics



In [ ]:
# 6 Ajout dans Prometheus 

# fichier de configuration Prometheus prometheus.yml :

#------code-----

# scrape_configs:
#   - job_name: "flask_app"
#     static_configs:
#       - targets: ["host.docker.internal:8000"]


#-----code------

# 'localhost:8000' uniquement depuis le conteneur Prometheus ne fonctionnerait pas,
# car localhost fait référence à Prometheus lui-même et non à notre machine hôte.
# Avec host.docker.internal, Docker permet au conteneur d’accéder aux services sur la machine hôte.

# http://localhost:9090/targets?search=&scrapePool=flask_app

# target doit etre 'UP' en vert



#     Exploration des métriques dans Prometheus :
#         Dans l'interface de Prometheus (http://localhost:9090), il faut aller dans l'onglet Graph.
#         Dans la barre de recherche de l'onglet Graph, on peut taper le nom d'une des métriques, comme recommendation_similarity_mean, et ensuite clic sur Execute.
#         On verra les valeurs actuelles et, en choisissant l'onglet Graph dessous, on peut voir l'évolution dans le temps.

#     Écrire des requêtes PromQL :
#         Prometheus utilise le langage PromQL pour interroger et filtrer les données. Par exemple :
#             recommendation_similarity_mean: pour afficher la dernière valeur de cette métrique.
#             rate(recommendation_similarity_mean[1m]): pour voir le taux de changement de cette métrique sur une minute.

# Ces requêtes peuvent être personnalisées pour obtenir des informations spécifiques, comme les tendances des recommandations ou les pics d'utilisation.


# On peut également zoomer en abscisse/ordonnée pour avoir un meilleur visuel global


In [ ]:
# On 

In [ ]:
# 7 Grafana , outil de monitoring 

# pour visualiser les métriques sous forme de graphiques de suivi en temps réel des recommandations.

# --> admin ET password 


# Visualisation dans Grafana

# Maintenant que Prometheus collecte les données, configurons Grafana pour visualiser ces métriques.

#     Accès à Grafana :  http://localhost:3000.


#     Pour Ajouter Prometheus comme source de données :

#         Faut Aller dans Configuration > Data Sources.
#         Ensuite Cliquer sur Add data source et sélectionnez Prometheus.

#         Entrez l'URL http://prometheus:9090 pour la source de données, car Grafana et Prometheus fonctionnent sur le même réseau Docker.
#         Cliquez sur Save & Test pour vérifier la connexion.

#    Créer un Dashboard pour les métriques :

#         aller dans Dashboards > New Dashboard > Add new panel.
#         après dans le champ de requête, on entre l'une des métriques, comme recommendation_similarity_mean.
#         Grafana va afficher automatiquement la métrique dans le graphique en temps réel.
#         pour ajouter d'autres panels pour les différentes métriques que l'on souhaite monitorer, 
#         comme recommendation_similarity_std ou recommendation_popularity_mean par exemple

#    Personnaliser le Dashboard :

#         On va pouvoir mmodifier les paramètres de visualisation (type de graphique, couleurs par exemple) pour rendre le dashboard plus lisible.
#         Et aussi ajouter des alertes pour être notifié pour certaines conditions (par exemple, si le score de similarité moyen 
#         descend en dessous d’un certain seuil).

#   Exemple d'alertes dans Grafana

#         Dans un panel, clic sur Edit et aller dans l’onglet Alert.
#         Configure une alerte pour envoyer une notification si une métrique dépasse un certain seuil.

#         On définit l’intervalle de vérification (par exemple, toutes les 1 minute) et la condition d’alerte 
#         (par exemple, recommendation_similarity_mean > 0.8).
#         Ajoutez un canal de notification (par e-mail, Slack, etc.) pour recevoir les alertes.

In [ ]:
# Attention car si on reste sur la meme page trop longtemps sans se deconnecter/connecter et qu'on re essaye d'utiliser l'appli le token sera surement périmé ,
# Les tokens sont configurés pour expirer apres un certain temps pour des raisons de sécurité 

In [ ]:
#Voir pour afficher les 3 métriques en meme temps sur Grafana